In [43]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm

In [13]:
urls = [
    'https://www.championat.com/football/_russiapl/tournament/3953/teams/',
    'https://www.championat.com/football/_russiapl/tournament/4103/teams/',
    'https://www.championat.com/football/_russia1d/tournament/3959/teams/',
    'https://www.championat.com/football/_russia2d/tournament/4613/teams/',
    'https://www.championat.com/football/_russia2d/tournament/4617/teams/',
    'https://www.championat.com/football/_russia2d/tournament/4619/teams/',
    'https://www.championat.com/football/_russia2d/tournament/4615/teams/'
]

In [ ]:
headers = {
    "Accept": "*/*",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 OPR/73.0.3856.344 (Edition Yx 03)"
}

In [23]:
teams_get = requests.get(url=urls[0], headers=headers)
soup = BeautifulSoup(teams_get.text)
all_ = soup.find_all('div', class_='teams-item')
teams = []
for url in urls:
    teams_get = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(teams_get.text)
    all_ = soup.find_all('div', class_='teams-item')
    for one in all_:
        link = 'https://www.championat.com' + one.find('a', class_='teams-item__link').get('href')
        teams.append({
            'team': one.find('div', class_='teams-item__name').text,
            'link': re.sub('result', 'players', link),
            'tournament': url
        })
teams

[{'team': 'Арсенал',
  'link': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/207511/players/',
  'tournament': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/'},
 {'team': 'Ахмат',
  'link': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/207513/players/',
  'tournament': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/'},
 {'team': 'Динамо М',
  'link': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/207515/players/',
  'tournament': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/'},
 {'team': 'Зенит',
  'link': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/207517/players/',
  'tournament': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/'},
 {'team': 'Краснодар',
  'link': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/207519/players/',
  'tournament': 'https://www.championat.com/football/_r

In [107]:
def clear(string):
    string = re.sub('\n', '', string)
    return string.strip()

In [72]:
players = []
for team in tqdm(teams):
    req = requests.get(team['link'], headers=headers)
    soup = BeautifulSoup(req.text)
    table_rows = soup.find('table', class_='table-responsive').find_all('tr')
    for row in table_rows[1:]:
        href = 'https://www.championat.com' + row.find('td', class_='_order_1').find('a').get('href')
        player_get = requests.get(href, headers=headers)
        sp = BeautifulSoup(player_get.text)
        tb = sp.find('table', class_='table-stripe').find_all('tr')
        li = sp.find('ul', class_='entity-header__facts js-entity-header-facts').find('div', text='Амплуа:').find_next().text
        ctr = re.sub('Гражданство:', '', li)
        players.append({
            'name': row.find('span', class_='table-item__name').text,
            'player link': href,
            'birthday': clear(row.find('td', class_='_order_4').text),
            'all matches': clear(tb[0].find('td', class_='_right-td').text),
            'all minutes': clear(tb[5].find('td', class_='_right-td').text),
            'country': clear(ctr),
            'team': team['team'],
            'team link': team['link'],
            'tournament': team['tournament'],
        })
        
players

100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [36:49<00:00, 19.72s/it]


[{'name': 'Григорий Григорьевич Кеваев ',
  'player link': 'https://www.championat.com/football/_russiapl/tournament/3953/players/315761/',
  'birthday': '\n                        26.11.2001                    ',
  'all matches': '\n\n',
  'all minutes': '\n\n',
  'country': '\n\n                                    Россия                            ',
  'team': 'Арсенал',
  'team link': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/207511/players/',
  'tournament': 'https://www.championat.com/football/_russiapl/tournament/3953/teams/'},
 {'name': 'Михаил Алексеевич Левашов ',
  'player link': 'https://www.championat.com/football/_russiapl/tournament/3953/players/36686/',
  'birthday': '\n                        04.10.1991                    ',
  'all matches': '\n3\n',
  'all minutes': '\n270\n',
  'country': '\n\n                                    Россия                            ',
  'team': 'Арсенал',
  'team link': 'https://www.championat.com/football/_rus

In [106]:
df = pd.DataFrame(players)
df.to_csv('full3.csv',index=False)
df.to_excel('full3.xlsx',index=False)


In [108]:
df

,name,player link,birthday,all matches,all minutes,country,team,team link,tournament
0,Григорий Григорьевич Кеваев,https://www.championat.com/football/_russiapl/...,26.11.2001,,,Россия,Арсенал,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russiapl/...
1,Михаил Алексеевич Левашов,https://www.championat.com/football/_russiapl/...,04.10.1991,3,270,Россия,Арсенал,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russiapl/...
2,Юрий Владимирович Лодыгин,https://www.championat.com/football/_russiapl/...,26.05.1990,,,Россия,Арсенал,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russiapl/...
3,Артур Эдуардович Нигматуллин,https://www.championat.com/football/_russiapl/...,17.05.1991,9,806,Россия,Арсенал,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russiapl/...
4,Андрей Евгеньевич Сорокин,https://www.championat.com/football/_russiapl/...,21.04.2001,,,Россия,Арсенал,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russiapl/...
...,...,...,...,...,...,...,...,...,...
4335,Хосе Саломон Рондон Хименес,https://www.championat.com/football/_russiapl/...,16.09.1989,10,743,Венесуэла,ЦСКА,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russia2d/...
4336,Фёдор Николаевич Чалов,https://www.championat.com/football/_russiapl/...,10.04.1998,27,1225,Россия,ЦСКА,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russia2d/...
4337,Илья Сергеевич Шкурин,https://www.championat.com/football/_russiapl/...,17.08.1999,9,361,Беларусь,ЦСКА,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russia2d/...
4338,Чидера Эджуке,https://www.championat.com/football/_russiapl/...,02.01.1998,25,1589,Нигерия,ЦСКА,https://www.championat.com/football/_russiapl/...,https://www.championat.com/football/_russia2d/...
